In [2]:
import pandas as pd
from sqlalchemy import create_engine
import openmeteo_requests

# Para manipular requisições e tentativas de reconexão
import requests_cache
from retry_requests import retry


In [3]:
# Configuração de cache e mecanismo de retentativa
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)


In [4]:
# Definição da localização e parâmetros para a requisição API
params = {
    "latitude": -22.9068,  # Latitude do Rio de Janeiro
    "longitude": -43.1729,  # Longitude do Rio de Janeiro
    "hourly": ["pressure_msl"],  # Solicitando a pressão atmosférica média ao nível do mar por hora
    "start": pd.Timestamp.now().strftime('%Y-%m-%d'),  # Data de início
    "end": (pd.Timestamp.now() + pd.Timedelta(days=7)).strftime('%Y-%m-%d'),  # Data final, 7 dias a partir de agora
    "timezone": "America/Sao_Paulo"  # Fuso horário local
}

In [5]:
# Realização da requisição à API
responses = openmeteo.weather_api("https://api.open-meteo.com/v1/forecast", params=params)
response = responses[0]  # Assumindo resposta de uma única localização


In [6]:
# Extração dos dados de pressão
hourly = response.Hourly()
pressure_values = hourly.Variables(0).ValuesAsNumpy()  # Supondo que a pressão seja a primeira variável solicitada

# Criação de um DataFrame
hourly_data = {
    "momento": pd.date_range(
        start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
        end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=hourly.Interval()),
        inclusive="left"
    ),
    "valor": pressure_values
}
df = pd.DataFrame(hourly_data)
df['id'] = df.index + 1


In [ ]:
# Ordenação dos dados pelo momento
df = df.sort_values('momento')

# Reordenando as colunas para corresponder à ordem: id, momento, valor
df = df[['id', 'momento', 'valor']]

# Conexão com o banco de dados usando SQLAlchemy
engine = create_engine('sqlite:///weather_data.db')  # Ajuste conforme a configuração de sua conexão

# Armazenamento dos dados na tabela
df.to_sql('previsao_pressao_atm', con=engine, if_exists='append', index=False)
print("Dados armazenados com sucesso!")

Dados armazenados com sucesso!


: 